# Autoregressive covariance structure 

We try to parameterize covariance Gamma only using two parameters rho and sigma2 as in the AR(1) structure.

For n = 50 this autoregressive model converges in 31 iterations using Quasi-Newton

1. First we need to modify the structure itself
2. Second modify the loglikelihood function 
3. Third modify gradient and hessian with respect to beta
4. ADD gradient and hessian with respect to AR(1) parameterization
5. Quasi-Newton works, we need to fix and make Newton work

In [10]:
using DataFrames, Random, GLM, GLMCopula, Test
using LinearAlgebra, BenchmarkTools, Revise

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 50
ρ = 0.5
σ2 = 0.1

V = zeros(n, n) # will store the AR(1) structure without sigma2

mean = 5

dist = Poisson

V = get_AR_cov(n, ρ, σ2, V)

# true Gamma
Γ = σ2 * V

50×50 Array{Float64,2}:
 0.1          0.05         0.025        …  3.55271e-16  1.77636e-16
 0.05         0.1          0.05            7.10543e-16  3.55271e-16
 0.025        0.05         0.1             1.42109e-15  7.10543e-16
 0.0125       0.025        0.05            2.84217e-15  1.42109e-15
 0.00625      0.0125       0.025           5.68434e-15  2.84217e-15
 0.003125     0.00625      0.0125       …  1.13687e-14  5.68434e-15
 0.0015625    0.003125     0.00625         2.27374e-14  1.13687e-14
 0.00078125   0.0015625    0.003125        4.54747e-14  2.27374e-14
 0.000390625  0.00078125   0.0015625       9.09495e-14  4.54747e-14
 0.000195313  0.000390625  0.00078125      1.81899e-13  9.09495e-14
 9.76563e-5   0.000195313  0.000390625  …  3.63798e-13  1.81899e-13
 4.88281e-5   9.76563e-5   0.000195313     7.27596e-13  3.63798e-13
 2.44141e-5   4.88281e-5   9.76563e-5      1.45519e-12  7.27596e-13
 ⋮                                      ⋱               
 3.63798e-13  7.27596e-13  1.45519e

In [11]:
vecd = [dist(mean) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Array{Array{Float64,1},1}:
 [7.0, 8.0, 7.0, 4.0, 8.0, 2.0, 6.0, 6.0, 6.0, 7.0  …  5.0, 3.0, 4.0, 6.0, 4.0, 5.0, 5.0, 6.0, 4.0, 7.0]
 [3.0, 7.0, 6.0, 7.0, 6.0, 6.0, 3.0, 3.0, 8.0, 5.0  …  3.0, 4.0, 5.0, 8.0, 2.0, 5.0, 4.0, 4.0, 8.0, 6.0]
 [6.0, 3.0, 3.0, 7.0, 4.0, 3.0, 6.0, 2.0, 1.0, 8.0  …  2.0, 4.0, 3.0, 4.0, 6.0, 3.0, 4.0, 4.0, 3.0, 5.0]
 [9.0, 4.0, 5.0, 3.0, 5.0, 5.0, 4.0, 12.0, 4.0, 5.0  …  4.0, 8.0, 7.0, 9.0, 5.0, 3.0, 8.0, 5.0, 4.0, 4.0]
 [5.0, 7.0, 4.0, 7.0, 4.0, 1.0, 7.0, 3.0, 5.0, 4.0  …  9.0, 7.0, 3.0, 1.0, 3.0, 5.0, 3.0, 6.0, 5.0, 5.0]
 [6.0, 6.0, 10.0, 2.0, 4.0, 2.0, 6.0, 7.0, 1.0, 5.0  …  2.0, 6.0, 3.0, 6.0, 3.0, 6.0, 4.0, 4.0, 4.0, 5.0]
 [5.0, 4.0, 6.0, 3.0, 4.0, 6.0, 0.0, 3.0, 3.0, 3.0  …  4.0, 6.0, 3.0, 3.0, 5.0, 1.0, 4.0, 1.0, 5.0, 10.0]
 [6.0, 4.0, 5.0, 7.0, 5.0, 8.0, 5.0, 3.0, 10.0, 1.0  …  5.0, 4.0, 5.0, 3.0, 4.0, 6.0, 5.0, 4.0, 7.0, 2.0]
 [4.0, 5.0, 8.0, 5.0, 5.0, 4.0, 7.0, 3.0, 3.0, 6.0  …  5.0, 2.0, 4.0, 7.0, 6.0, 5.0, 6.0, 5.0, 7.0, 4.0]
 [4.0, 6.0

In [12]:
Random.seed!(1234)

d = Poisson()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaARObs{T, D, Link}}(undef, N)

for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    gcs[i] = GLMCopulaARObs(y, X, d, link)
end

gcm = GLMCopulaARModel(gcs);

In [13]:
initialize_model!(gcm)
@show gcm.β
@show exp.(gcm.β);

initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -1.1093804486616538e6 39999
2 -1.1093804486616538e6 -1.1093804486616538e6 39999
gcm.β = [1.6129668783028606]
exp.(gcm.β) = [5.017676]


In [14]:
fill!(gcm.ρ, 0.1)
fill!(gcm.σ2, 1.0)
loglikelihood!(gcm, true, true)

gcm2 = deepcopy(gcm);

In [15]:
@time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100, warm_start_init_point="yes", hessian_approximation = "limited-memory"))

gcm.θ = [1.6129668783028606, 0.1, 1.0]
gcm.∇θ = [-9153.69882469465, 208.66567501442762, -4.618677955911121]
gcm.θ = [1.6129668783028606, 0.1, 1.0]
gcm.∇θ = [-9153.69882469465, 208.66567501442762, -4.618677955911121]
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0


  31  1.1092194e+06 0.00e+00 4.33e-10 -11.0 1.52e-08    -  1.00e+00 1.00e+00f  1

Number of Iterations....: 31

                                   (scaled)                 (unscaled)
Objective...............:   1.2117718046756576e+04    1.1092194144257682e+06
Dual infeasibility......:   4.3285225643172636e-10    3.9621991909655208e-08
Constraint violation....:   0.0000000000000000e+00    0.0000000000000000e+00
Complementarity.........:   9.9999999999999396e-12    9.1536988246945941e-10
Overall NLP error.......:   4.3285225643172636e-10    3.9621991909655208e-08


Number of objective function evaluations             = 118
Number of objective gradient evaluations             = 32
Number of equality constraint evaluations            = 0
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian evaluations   = 0
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 0
Total CPU secs in IPOPT (w/o

-1.1092194144257682e6

In [16]:
@show gcm.∇θ

gcm.∇θ = [-3.962199190965521e-8, 1.5657526386547715e-9, 1.6555707915699713e-8]


3-element Array{Float64,1}:
 -3.962199190965521e-8
  1.5657526386547715e-9
  1.6555707915699713e-8

In [17]:
@show gcm.θ

gcm.θ = [1.6102423786490345, 0.525419788411791, 0.08789570936927205]


3-element Array{Float64,1}:
 1.6102423786490345
 0.525419788411791
 0.08789570936927205

In [18]:
@show loglikelihood!(gcm, true, true)

loglikelihood!(gcm, true, true) = -1.1092194144257682e6


-1.1092194144257682e6